In [33]:
import tensorflow as tf
import os
import numpy as np
import time
import datetime

from PIL import Image
from tensorflow.keras import layers

In [34]:
IMAGE_SIZE = 64
IMAGE_CHANNELS = 3
BATCH_SIZE = 64
DATASET_DIR = 'wikiart'
#CLASSES = ["Impressionism", "Baroque", "Expressionism", "Art_Nouveau_Modern", "Romanticism"]
CLASSES = ["Baroque"]

In [35]:
def preprocess_image(image):
    image = np.array(image)
    image /= 127.5
    image -= 1
    return image

def get_dataset():
    img_gen = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocess_image)
    generator_method = img_gen.flow_from_directory(
        DATASET_DIR, 
        target_size=(IMAGE_SIZE, IMAGE_SIZE), 
        classes = CLASSES, 
        batch_size = 1)
    
    return tf.data.Dataset.from_generator(
        lambda: generator_method,
        output_types=(tf.float32, tf.float32), 
        output_shapes=([1, IMAGE_SIZE, IMAGE_SIZE, IMAGE_CHANNELS], [1, len(CLASSES)])
    )
    

In [36]:
BUFFER_SIZE = 500
STEPS_PER_EPOCH = 4240  // BATCH_SIZE
train_dataset = get_dataset().shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

Found 4240 images belonging to 1 classes.


In [37]:
NOISE_DIMS = 100
LEAKY_RELU_ALPHA = 0.2

def create_generator(): 
    weight_initializer = tf.random_normal_initializer(stddev=0.02)
    
    def conv2d_transponse_layer(input, filters, kernel=5):
        conv = layers.Conv2DTranspose(filters, (kernel, kernel), strides=(2, 2), padding='same', use_bias=False, 
                                  kernel_initializer=weight_initializer)(input)
        batch = layers.BatchNormalization()(conv)
        return layers.LeakyReLU(LEAKY_RELU_ALPHA)(batch)
    
    noise = layers.Input(shape=(NOISE_DIMS,))
    label = layers.Input(shape=(len(CLASSES),))
    input = layers.Concatenate()([noise, label])
    
    dense = layers.Dense(4*4*1024, use_bias=False, kernel_initializer=weight_initializer)(input)    
    reshaped = layers.Reshape((4, 4, 1024))(dense)
    
    conv1 = conv2d_transponse_layer(reshaped, filters=512)
    conv2 = conv2d_transponse_layer(conv1, filters=256)
    conv3 = conv2d_transponse_layer(conv2, filters=128)
    #conv4 = conv2d_transponse_layer(conv3, filters=256)
    
    out = layers.Conv2DTranspose(IMAGE_CHANNELS, (3, 3), strides=(2, 2), padding='same', use_bias=False, 
                                 activation='tanh')(conv3)
    print(out.shape)
    return tf.keras.Model(inputs=[noise, label], outputs=out)

In [38]:
def create_discriminator(): 
    weight_initializer = tf.random_normal_initializer(stddev=0.02)
    
    def conv2d_layer(input, filters, stride=2):
        conv = layers.Conv2D(filters, (5, 5), strides=(stride, stride), padding='same',
                             kernel_initializer= weight_initializer)(input)
        batch = layers.BatchNormalization()(conv)
        relu = layers.LeakyReLU(LEAKY_RELU_ALPHA)(batch)
        dropout = layers.Dropout(0.25)
        return relu
    
    image = layers.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, IMAGE_CHANNELS))
    label = layers.Input(shape=(len(CLASSES),))
    
    #random noise
    noise = layers.GaussianNoise(0.2)(image)
    
    conv = layers.Conv2D(16, (5, 5), strides=(2, 2), padding='same',
                             kernel_initializer= weight_initializer)(noise)
    relu = layers.LeakyReLU(LEAKY_RELU_ALPHA)(conv)
    drop = layers.Dropout(0.25)(relu)
    
    conv2 = conv2d_layer(drop, filters=32)
    conv3 = conv2d_layer(conv2, filters=64)
    conv4 = conv2d_layer(conv3, filters=128)
    print(conv4.shape)
    
    img_out = layers.Flatten()(conv)
    merged = layers.Concatenate()([img_out, label])
    out = layers.Dense(1, activation='sigmoid')(merged)

    return tf.keras.Model(inputs=[image, label], outputs=out)

In [39]:
LEARNING_RATE = 0.0002

cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_output, fake_output):
    #real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    #fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    
    #label smoothing
    real_loss = cross_entropy((tf.random.uniform(real_output.shape.as_list(), minval=0.7, maxval=1)), real_output)
    fake_loss = cross_entropy(tf.random.uniform(fake_output.shape.as_list(), minval=0.0, maxval=0.3), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    #label smoothing
    return cross_entropy(tf.random.uniform(fake_output.shape, minval=0.7, maxval=1.2), fake_output)
    #return cross_entropy(tf.ones_like(fake_output), fake_output)

generator_optimizer = tf.keras.optimizers.Adam(LEARNING_RATE, beta_1=0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(LEARNING_RATE, beta_1=0.5)

generator = create_generator()
discriminator = create_discriminator()

(None, 64, 64, 3)
(None, 4, 4, 128)


In [40]:
def train_step(images, labels, step):
    noise = tf.random.normal([BATCH_SIZE, NOISE_DIMS])
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator([noise, labels], training=True)
        
        real_output = discriminator([images, labels], training=True)    
        fake_output = discriminator([generated_images, labels], training=True)

        disc_loss = discriminator_loss(real_output, fake_output)
        gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
        discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
        
        #gen_loss_metric(gen_loss)
        #disc_loss_metric(disc_loss)
        
        discriminator.trainable = False
        fake_output2 = discriminator([generated_images, labels], training=False)
        gen_loss = generator_loss(fake_output2) 
        gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
        
        if(step % 20 == 0):
            print('Step {}, Gen loss {}, Disc loss {}, Dx {}, DGz1 {}, DGz2 {} '.format(step, gen_loss.numpy(), 
                                                                               disc_loss.numpy(),
                                                                               np.mean(real_output.numpy()), 
                                                                               np.mean(fake_output.numpy()),
                                                                               np.mean(fake_output2.numpy())))
        
        generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
        discriminator.trainable = True


In [41]:
seed = tf.random.normal([1, NOISE_DIMS])
label = tf.constant([[1]])

def train(dataset, epochs):
  for epoch in range(epochs):
    print("Starting epoch...")
    start = time.time()
    
    step = 1
    for image_batch, label_batch in dataset:
      print("Step ", step)
      train_step(tf.squeeze(image_batch), tf.squeeze(label_batch), step)
      if(step == STEPS_PER_EPOCH): 
        break
      step = step + 1

    
    res = generator([seed, label], training=False).numpy()

    for pic in res:
        pic = pic * 127.5 + 127.5
        img = Image.fromarray(pic.astype('uint8'))
        img.save("pic{}.png".format(epoch))
    
    #with train_summary_writer.as_default():
     #   tf.summary.scalar('Gen loss', gen_loss_metric.result(), step=epoch)
     #   tf.summary.scalar('Disc loss', disc_loss_metric.result(), step=epoch)
        
    #template = 'Epoch {}, Gen loss: {}, Disc loss: {}'
    #print(template.format(epoch+1,gen_loss_metric.result(), disc_loss_metric.result()))
    print('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

In [42]:
EPOCHS = 20
train(train_dataset, EPOCHS)

Starting epoch...
Step  1
Step  2
Step  3
Step  4
Step  5
Step  6
Step  7
Step  8
Step  9
Step  10
Step  11
Step  12
Step  13
Step  14
Step  15
Step  16
Step  17
Step  18
Step  19
Step  20
Step 20, Gen loss 0.3966459035873413, Disc loss 1.618373155593872, Dx 0.712865948677063, DGz1 0.9490775465965271, DGz2 0.9467740058898926 
Step  21
Step  22
Step  23
Step  24
Step  25
Step  26
Step  27
Step  28
Step  29
Step  30
Step  31
Step  32
Step  33
Step  34
Step  35
Step  36
Step  37
Step  38
Step  39
Step  40
Step 40, Gen loss 0.3474189341068268, Disc loss 1.6335217952728271, Dx 0.8344216346740723, DGz1 0.9848053455352783, DGz2 0.9848477244377136 
Step  41
Step  42
Step  43
Step  44
Step  45
Step  46
Step  47
Step  48
Step  49
Step  50
Step  51
Step  52
Step  53
Step  54
Step  55
Step  56
Step  57
Step  58
Step  59
Step  60
Step 60, Gen loss 0.33042484521865845, Disc loss 1.6421608924865723, Dx 0.8953465223312378, DGz1 0.9944698810577393, DGz2 0.9946238994598389 
Step  61
Step  62
Step  63
St

KeyboardInterrupt: 

In [212]:
res = generator([tf.random.normal([1, NOISE_DIMS]), tf.constant([[1]])], training=False).numpy()

for pic in res:
    pic = pic * 127.5 + 127.5
    img = Image.fromarray(pic.astype('uint8'))
    img.show()